In [ ]:
import sqlite3 as lite
from monitoring import DB_PATH, SENSORS, SensorDB



In [ ]:

import pandas as pd

# scp axlan@192.168.1.110:~/src/cat_chair/controller/*.sqlite controller/
con = lite.connect(f'file:{DB_PATH}?mode=ro', uri=True)
df = pd.read_sql_query("SELECT * from sensor_data", con, parse_dates=['timestamp'])
con.close()

In [ ]:
import plotly.express as px

df_disp = df[df['value'] >= 0]
df_disp['value'].replace(0, 0.1, inplace=True)
df_disp['timestamp'] = df_disp['timestamp'].dt.tz_localize(tz='UTC')
df_disp['timestamp'] = df_disp['timestamp'].dt.tz_convert(tz='US/Pacific')

#
# print(df)

fig = px.scatter(df_disp, x="timestamp", y="value", log_y=True, color="name")
#fig = px.scatter(df_disp, x="timestamp", y="value", color="name")
fig.show()
# 1/9 sunny, no chair sensor until late in the day
# 1/10 mix of sun and clouds. Sun briefly passed over chair. Sensor spacing activated back briefly, then front
# 1/11 Mostly sunny day, started with the chair in place, then moved once the be in the sun again.
# 1/12 Mostly cloudy day, moved chair when brief spot of sun occured


In [ ]:
import plotly.graph_objects as go

WINDOW_THRESHOLD = 20000

def get_vals(name):
    return df_disp[df_disp['name'].str.match(name)].set_index('timestamp')['value']

def combine(left, right, suffixes=None):
    return pd.merge_asof(left, right, left_index=True, right_index=True,direction='nearest',
                         suffixes=suffixes, tolerance=pd.Timedelta(0.5,'sec') ).dropna()

window_series = get_vals('window')
front_series = get_vals('chair_front')
back_series = get_vals('chair_back')

filtered = combine(window_series, front_series, ['_window','_front'])
filtered = combine(filtered, back_series)
filtered.rename(columns={"value": "value_back"}, inplace=True)

filtered = filtered[filtered['value_window'] > WINDOW_THRESHOLD]

fig = go.Figure()

fig.add_trace(go.Scatter(x=filtered.index, y=filtered['value_front'] - filtered['value_window'], mode='markers',name='front'))
fig.add_trace(go.Scatter(x=filtered.index, y=filtered['value_back'] - filtered['value_window'], mode='markers',name='back'))

fig.show()


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=filtered.index, y=filtered['value_front'] / filtered['value_window'], mode='markers',name='front'))
fig.add_trace(go.Scatter(x=filtered.index, y=filtered['value_back'] / filtered['value_window'], mode='markers',name='back'))

fig.show()
